In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/zach/pbp-2017.csv")

In [10]:
df.columns.values.tolist()


['GameId',
 'GameDate',
 'Quarter',
 'Minute',
 'Second',
 'OffenseTeam',
 'DefenseTeam',
 'Down',
 'ToGo',
 'YardLine',
 'Unnamed: 10',
 'SeriesFirstDown',
 'Unnamed: 12',
 'NextScore',
 'Description',
 'TeamWin',
 'Unnamed: 16',
 'Unnamed: 17',
 'SeasonYear',
 'Yards',
 'Formation',
 'PlayType',
 'IsRush',
 'IsPass',
 'IsIncomplete',
 'IsTouchdown',
 'PassType',
 'IsSack',
 'IsChallenge',
 'IsChallengeReversed',
 'Challenger',
 'IsMeasurement',
 'IsInterception',
 'IsFumble',
 'IsPenalty',
 'IsTwoPointConversion',
 'IsTwoPointConversionSuccessful',
 'RushDirection',
 'YardLineFixed',
 'YardLineDirection',
 'IsPenaltyAccepted',
 'PenaltyTeam',
 'IsNoPlay',
 'PenaltyType',
 'PenaltyYards']

In [192]:
df.IsTouchdown

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      1
27      0
28      0
29      0
       ..
5393    0
5394    0
5395    0
5396    0
5397    0
5398    0
5399    0
5400    0
5401    0
5402    0
5403    0
5404    0
5405    0
5406    0
5407    0
5408    0
5409    0
5410    0
5411    0
5412    0
5413    0
5414    0
5415    0
5416    0
5417    0
5418    0
5419    0
5420    0
5421    0
5422    0
Name: IsTouchdown, Length: 5423, dtype: int64

In [167]:
play_description= df['Description'].loc[2890]
play_type = df['PlayType'].loc[2]

In [168]:
play_description

'(12:10) (SHOTGUN) 12-A.RODGERS SACKED AT GB 40 FOR -6 YARDS (72-M.BENNETT).'

In [199]:
import re

def format_return(series, player_info):
    play_type = series['PlayType']
    yards = series['Yards']
    
    # Don't count interception return yardage for the offense
    if series['IsInterception'] == 1:
        yards = 0
        # Put the interception in the qb stats
        result = "INTERCEPTION"
    elif series['IsIncomplete'] == 1:
        result = "INCOMPLETE"
    else: result = "RECEPTION"
        
    if series['IsTouchdown'] == 1:
        touchdown = True
    else:
        touchdown = False
    
    if play_type == "PASS":
        return {"play_type":play_type, "yards":yards, "touchdown": touchdown, "ball_carrier_number":player_info[0], "ball_carrier_name":player_info[1], "receiver_number": player_info[2], "reciever_name": player_info[3], "result": result}

    if play_type == "RUSH":
        return {"play_type":play_type, "yards":yards, "touchdown": touchdown, "ball_carrier_number":player_info[0], "ball_carrier_name":player_info[1]}
    
def extract_runner(series):
    rush_regex = "\s\d{1,2}-\w+.\w+\s"
    pass_regex = "\s\d{1,2}-\w+.\w+\s(?=PASS)"
    receive_regex = "(?<=TO)\s\d{1,2}-\w+.\s*\w+|(?<=FOR)\s\d{1,2}-\w+.\s*\w+"
    
    if series['PlayType'] == 'PASS':
        result = re.search(pass_regex, series['Description'])
        result2 = re.search(receive_regex, series['Description'])
        try:
            player_info = result.group(0).strip().split('-')
            receiver_info = result2.group(0).strip().split('-')
            player_info.append(receiver_info[0])
            player_info.append(receiver_info[1])
            return(format_return(series, player_info))
        except Exception as e:
            print e
            print series["Description"]
    
    if series['PlayType'] == 'RUSH':
        result = re.search(rush_regex, series['Description'])
        try:
            player_info = result.group(0).strip().split('-')
            return(format_return(series, player_info))
        except Exception as e:
            print e
    
    
a = df.apply(lambda x: extract_runner(x), axis=1)

'NoneType' object has no attribute 'group'
(1:55) (SHOTGUN) 4-D.PRESCOTT PASS INCOMPLETE SHORT RIGHT.
'NoneType' object has no attribute 'group'
(10:33) (SHOTGUN) 13-T.SIEMIAN PASS INCOMPLETE DEEP RIGHT.
'NoneType' object has no attribute 'group'
(5:44) 8-M.MARIOTA PASS INCOMPLETE SHORT RIGHT. MARIOTA THREW THE BALL AWAY.
'NoneType' object has no attribute 'group'
(10:24) (SHOTGUN) 8-M.MARIOTA PASS INCOMPLETE SHORT LEFT. MARIOTA THREW THE BALL AWAY.
'NoneType' object has no attribute 'group'
(:10) (SHOTGUN) 8-M.MARIOTA PASS INCOMPLETE SHORT RIGHT.
'NoneType' object has no attribute 'group'
(:40) (NO HUDDLE, SHOTGUN) 8-M.MARIOTA PASS INCOMPLETE SHORT RIGHT.
'NoneType' object has no attribute 'group'
(9:01) (SHOTGUN) 1-C.NEWTON PASS INCOMPLETE SHORT LEFT.
'NoneType' object has no attribute 'group'
(9:48) 14-A.DALTON PASS INCOMPLETE DEEP RIGHT. {THROWN AWAY}
'NoneType' object has no attribute 'group'
(:26) (SHOTGUN) 11-A.SMITH PASS INCOMPLETE SHORT LEFT [58-J.HICKS]. A.SMITH INTENTIONALLY

In [200]:
b= a.apply(pd.Series)

In [211]:
b.iloc[:80]

,ball_carrier_name,ball_carrier_number,play_type,receiver_number,reciever_name,result,touchdown,yards
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B.POWELL,29,RUSH,NaN,NaN,NaN,False,1.0
2,M.LYNCH,24,RUSH,NaN,NaN,NaN,False,5.0
3,K.COUSINS,8,PASS,85,V.DAVIS,INCOMPLETE,False,0.0
4,C.MCCAFFREY,22,RUSH,NaN,NaN,NaN,False,3.0
5,C.ANDERSON,22,RUSH,NaN,NaN,NaN,False,0.0
6,D.FOREMAN,27,RUSH,NaN,NaN,NaN,False,4.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,J.WINSTON,3,PASS,34,C.SIMS,INCOMPLETE,False,0.0


In [100]:
test = "(2:51) 9-M.LYNCH LEFT TACKLE TO TEN 7 FOR 5 YARDS (59-W.WOODYARD)."
import re
result = re.search("\s\d{1,2}-\w{1}.\w+\s", test)


In [106]:
type(b)

pandas.core.frame.DataFrame

In [41]:
result.group(0).strip().split("-")

['9', 'M.LYNCH']

In [204]:
b.to_json()

'{"ball_carrier_name":{"0":null,"1":"B.POWELL","2":"M.LYNCH","3":"K.COUSINS","4":"C.MCCAFFREY","5":"C.ANDERSON","6":"D.FOREMAN","7":null,"8":null,"9":"J.WINSTON","10":null,"11":"C.KEENUM","12":"R.WILSON","13":"T.RIDDICK","14":null,"15":null,"16":"E.MANNING","17":"S.VEREEN","18":"E.MANNING","19":"P.PERKINS","20":"E.MANNING","21":"R.KELLEY","22":"K.COUSINS","23":"S.PERINE","24":"S.PERINE","25":null,"26":"C.THOMPSON","27":"R.KELLEY","28":"R.KELLEY","29":"C.THOMPSON","30":"S.PERINE","31":"R.KELLEY","32":null,"33":"K.COUSINS","34":"K.COUSINS","35":"R.KELLEY","36":"K.COUSINS","37":null,"38":"R.KELLEY","39":"K.COUSINS","40":null,"41":"T.MONTGOMERY","42":"A.RODGERS","43":"A.RODGERS","44":"T.MONTGOMERY","45":"A.RODGERS","46":"T.MONTGOMERY","47":"A.RODGERS","48":null,"49":"D.FREEMAN","50":"T.COLEMAN","51":"T.GABRIEL","52":"M.RYAN","53":"D.FREEMAN","54":null,"55":null,"56":null,"57":"R.WILSON","58":"R.WILSON","59":"C.CARSON","60":null,"61":"T.GURLEY","62":"J.GOFF","63":null,"64":"K.COUSINS","65":